# Day 14

## Part 1

We are low on fuel!
Luckyly there are raw materials on the plante next to us.

Given some chemical reactions `X1*A1, X2*A2, ..., XN*AN => Y*B`, we need to compute how many ORE (the raw material) we need to produce some fuel.

For example, given the reactions:
```
10 ORE => 10 A
1 ORE => 1 B
7 A, 1 B => 1 C
7 A, 1 C => 1 D
7 A, 1 D => 1 E
7 A, 1 E => 1 FUEL
```

We need 31 ORE to produce 1 FUEL.

In [1]:
import re

component_re = re.compile(r"(\d+) ([A-Z]+)")

def parse_reactions(input_):
    reactions = {}
    for line in input_.splitlines():
        if not line:
            continue

        matches = component_re.findall(line)
        result_match = matches.pop()
        
        result_quantity = int(result_match[0])
        result_component = result_match[1]
        
        components = [
            {"quantity": int(m[0]), "component": m[1]}
            for m in matches
        ]
        reactions[result_component] = {
            "quantity": result_quantity,
            "components": components,
        }
        
    return reactions

In [2]:
from pprint import pprint

reactions = parse_reactions("""
10 ORE => 10 A
1 ORE => 1 B
7 A, 1 B => 1 C
7 A, 1 C => 1 D
7 A, 1 D => 1 E
7 A, 1 E => 1 FUEL
""")

pprint(reactions)

{'A': {'components': [{'component': 'ORE', 'quantity': 10}], 'quantity': 10},
 'B': {'components': [{'component': 'ORE', 'quantity': 1}], 'quantity': 1},
 'C': {'components': [{'component': 'A', 'quantity': 7},
                      {'component': 'B', 'quantity': 1}],
       'quantity': 1},
 'D': {'components': [{'component': 'A', 'quantity': 7},
                      {'component': 'C', 'quantity': 1}],
       'quantity': 1},
 'E': {'components': [{'component': 'A', 'quantity': 7},
                      {'component': 'D', 'quantity': 1}],
       'quantity': 1},
 'FUEL': {'components': [{'component': 'A', 'quantity': 7},
                         {'component': 'E', 'quantity': 1}],
          'quantity': 1}}


In [3]:
from collections import defaultdict
from math import ceil

def compute_quantities(wanted_component, wanted_quantity, reactions, storage=None):
    """Return the quantity of ORE needed to producte a component."""
    #print(wanted_component, wanted_quantity, dict(storage or {}))
    if wanted_component == "ORE":
        # It's free
        return wanted_quantity, storage

    if storage is None:
        storage = defaultdict(lambda: 0)

    recipe = reactions[wanted_component]
    components = recipe["components"]

    ore_quantity = 0
    quantity_mult = ceil(wanted_quantity / recipe["quantity"])

    for component in components:
        needed_quantity =  quantity_mult * component["quantity"]
        
        
        if storage[component["component"]] >= needed_quantity:
            storage[component["component"]] -= needed_quantity
            continue
        else:
            needed_quantity -= storage[component["component"]]
            storage[component["component"]] = 0

        component_ore_quantity, storage = compute_quantities(
            component["component"],
            needed_quantity,
            reactions,
            storage,
        )
        
        ore_quantity += component_ore_quantity
        
    total_produced = quantity_mult * recipe["quantity"]
    if total_produced > wanted_quantity:
        storage[wanted_component] += total_produced - wanted_quantity
        
    return ore_quantity, storage

In [4]:
def compute_fuel(reactions):
    q, _ = compute_quantities("FUEL", 1, reactions)
    return q

In [5]:
compute_fuel(reactions)

31

In [6]:
# should return 165
compute_fuel(parse_reactions("""
9 ORE => 2 A
8 ORE => 3 B
7 ORE => 5 C
3 A, 4 B => 1 AB
5 B, 7 C => 1 BC
4 C, 1 A => 1 CA
2 AB, 3 BC, 4 CA => 1 FUEL
"""))

165

In [7]:
# should return 13312
compute_fuel(parse_reactions("""
157 ORE => 5 NZVS
165 ORE => 6 DCFZ
44 XJWVT, 5 KHKGT, 1 QDVJ, 29 NZVS, 9 GPVTF, 48 HKGWZ => 1 FUEL
12 HKGWZ, 1 GPVTF, 8 PSHF => 9 QDVJ
179 ORE => 7 PSHF
177 ORE => 5 HKGWZ
7 DCFZ, 7 PSHF => 2 XJWVT
165 ORE => 2 GPVTF
3 DCFZ, 7 NZVS, 5 HKGWZ, 10 PSHF => 8 KHKGT
"""))

13312

In [8]:
# 180967
compute_fuel(parse_reactions("""
2 VPVL, 7 FWMGM, 2 CXFTF, 11 MNCFX => 1 STKFG
17 NVRVD, 3 JNWZP => 8 VPVL
53 STKFG, 6 MNCFX, 46 VJHF, 81 HVMC, 68 CXFTF, 25 GNMV => 1 FUEL
22 VJHF, 37 MNCFX => 5 FWMGM
139 ORE => 4 NVRVD
144 ORE => 7 JNWZP
5 MNCFX, 7 RFSQX, 2 FWMGM, 2 VPVL, 19 CXFTF => 3 HVMC
5 VJHF, 7 MNCFX, 9 VPVL, 37 CXFTF => 6 GNMV
145 ORE => 6 MNCFX
1 NVRVD => 8 CXFTF
1 VJHF, 6 MNCFX => 4 RFSQX
176 ORE => 6 VJHF
"""))

180697

In [9]:
# 2210736
compute_fuel(parse_reactions("""
171 ORE => 8 CNZTR
7 ZLQW, 3 BMBT, 9 XCVML, 26 XMNCP, 1 WPTQ, 2 MZWV, 1 RJRHP => 4 PLWSL
114 ORE => 4 BHXH
14 VRPVC => 6 BMBT
6 BHXH, 18 KTJDG, 12 WPTQ, 7 PLWSL, 31 FHTLT, 37 ZDVW => 1 FUEL
6 WPTQ, 2 BMBT, 8 ZLQW, 18 KTJDG, 1 XMNCP, 6 MZWV, 1 RJRHP => 6 FHTLT
15 XDBXC, 2 LTCX, 1 VRPVC => 6 ZLQW
13 WPTQ, 10 LTCX, 3 RJRHP, 14 XMNCP, 2 MZWV, 1 ZLQW => 1 ZDVW
5 BMBT => 4 WPTQ
189 ORE => 9 KTJDG
1 MZWV, 17 XDBXC, 3 XCVML => 2 XMNCP
12 VRPVC, 27 CNZTR => 2 XDBXC
15 KTJDG, 12 BHXH => 5 XCVML
3 BHXH, 2 VRPVC => 7 MZWV
121 ORE => 7 VRPVC
7 XCVML => 6 RJRHP
5 BHXH, 4 VRPVC => 5 LTCX
"""))

2210736

In [10]:
%%time

reactions = parse_reactions("""
4 NZGF => 6 WBMZG
20 FWMN, 2 QTMF, 5 FMVDV, 1 CVBPJ, 2 KVJK, 20 XSTBX, 7 NBFS => 5 SHPSF
7 LVQM => 5 NXDHX
1 FNDMP, 1 QZJV, 12 RMTG => 7 JBFW
10 GKVF, 1 NXDHX => 8 NZGF
12 QZJV => 8 RSMC
8 RWTD => 7 NBFS
4 CZGXS, 25 QTMF, 2 PHFQB => 3 BWQN
3 WQZD => 9 CTZKV
2 DCTQ, 18 CTZKV => 4 QLHZW
31 QLHZW, 11 FNDMP => 6 WFDXN
8 RLQC => 2 ZPJS
2 SWSQG, 13 CVBPJ => 9 DWCND
7 PBXB, 6 HKSWM, 4 BDPC, 4 KVJK, 2 ZLGKH, 9 LXFG, 1 ZPJS => 4 SWCWH
6 QZJV => 7 RLQC
3 QZJV, 11 MRQHX, 15 GKVF => 4 FMVDV
3 NXDHX, 1 GKNQL => 3 VMDS
1 VMDS => 2 RHSQ
13 GKNQL, 4 NXDHX, 2 GKVF => 8 MRQHX
4 PVRN => 2 WBSL
2 CVBPJ => 9 PVRN
3 FNDMP => 9 BZKC
180 ORE => 6 FWMN
13 DCTQ, 2 RHSQ => 5 CVBPJ
1 DWCND, 12 BZKC, 2 WBRBV => 6 HTLZ
1 LMGL, 11 XDVL, 7 DWCND => 5 ZLGKH
3 FMFTD => 3 HKSWM
1 FNDMP, 5 RMTG, 3 QLHZW => 9 CZGXS
7 DCTQ => 3 FNDMP
1 SHPSF, 2 SWCWH, 40 WFDXN, 67 WBMZG, 53 WBSL, 2 CQJDJ, 41 BWQN, 12 GMQVW, 48 PDRJ, 42 RSMC => 1 FUEL
3 VMDS, 1 BHRZ => 9 DCTQ
22 DCTQ, 4 NZGF => 7 RMTG
29 RWTD, 3 FMFTD => 5 LMGL
12 WBRBV, 13 PDRJ, 36 RSRG => 4 LXFG
1 SWSQG, 2 NLPB => 3 WBRBV
7 HTKLM, 8 CTZKV => 2 RWTD
4 BQXL, 1 FWMN => 9 GKNQL
4 WFDXN => 9 HTKLM
2 XDVL => 5 QTMF
1 PHFQB, 21 LMGL, 2 SWSQG => 7 GMQVW
23 CZGXS, 11 FMVDV => 3 PDRJ
1 DWCND, 1 NPMXR, 1 RSRG, 1 JBFW, 12 VXWKZ, 9 KVJK => 4 CQJDJ
106 ORE => 4 BQXL
4 PHFQB => 8 NPMXR
1 GKNQL => 8 WQZD
6 BDPC => 2 PHFQB
1 DWCND => 7 PBXB
1 RSMC, 1 PDRJ => 8 SWSQG
1 LVQM => 4 BHRZ
7 CVBPJ, 1 SWSQG, 1 NLPB => 2 VXWKZ
1 BHRZ, 1 JBFW => 6 XDVL
12 LMGL, 8 RWTD => 4 XSTBX
4 RSMC => 6 BDPC
1 BHRZ, 5 NXDHX => 3 GKVF
6 FMVDV, 6 VXWKZ, 37 CVBPJ => 5 KVJK
7 NLPB, 3 HTLZ => 4 RSRG
1 PDRJ => 1 FMFTD
6 RHSQ, 1 NZGF => 5 QZJV
127 ORE => 3 LVQM
3 RHSQ, 2 RLQC, 1 WFDXN => 1 NLPB
""")

compute_fuel(reactions)

CPU times: user 1.03 ms, sys: 7 μs, total: 1.03 ms
Wall time: 1.05 ms


857266

## Part 2

We want to know how much fuel we can produce with 1 trilion (1000000000000) ORE.

It is probably totally sub optimal, but I would process by guessing.

In [11]:
1000000000000 / 857266

1166499.0796322261

In [12]:
q, _ = compute_quantities("FUEL", 1166499, reactions)
q

543897956844

In [13]:
q, _ = compute_quantities("FUEL", 2144702, reactions)
q

999999752281

In [14]:
q, _ = compute_quantities("FUEL", 2144703, reactions)
q

1000000280269